In [1]:
import numpy as np
import cv2
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
import matplotlib
import time
import csv
import easygui
import serial
import time
from matplotlib.widgets import Slider, Button, TextBox
#from matplotlib.widgets import TextBox

In [2]:
def mypause(interval):
    manager = plt._pylab_helpers.Gcf.get_active()
    if manager is not None:
        canvas = manager.canvas
        if canvas.figure.stale:
            canvas.draw_idle()        
        plt.show(block=False)
        canvas.start_event_loop(interval)
    else:
        time.sleep(interval)
    return 0

In [3]:
def control(command):
    port = "COM4"
    ser = serial.Serial(port, 9600, timeout=1)
    ser.baudrate = 9600
    ser.bytesize = serial.EIGHTBITS #number of bits per bytes
    ser.parity = serial.PARITY_NONE #set parity check: no parity
    ser.stopbits = serial.STOPBITS_ONE #number of stop bits
    ser.timeout = 1            #non-block read
    ser.xonxoff = False     #disable software flow control
    ser.rtscts = False     #disable hardware (RTS/CTS) flow control
    ser.dsrdtr = False       #disable hardware (DSR/DTR) flow control
    ser.writeTimeout = 2     #timeout for write
    ser.close()
    ser.open()
    command=command.lower()
    if command=='start':
        DATA='4'.encode('utf-8')
    elif command=='down':
        DATA='3'.encode('utf-8')
    elif command=='up':
        DATA='2'.encode('utf-8')
    elif command=='pause':
        DATA='1'.encode('utf-8')
    else:
        print('illegal command, enter up, down, start, pause')
        return None
    if ser.isOpen():
        ser.flushInput() #flush input buffer, discarding all its contents
        ser.flushOutput()#flush output buffer, aborting current output
        ser.write(DATA)
        #print(DATA, '  written')
        time.sleep(0.5)  #give the serial port sometime to receive the data
        numOfLines = 0
    ser.close()
    return None
def update(val):
    global SETTEMP
    val=val.replace(',','.')
    try:
        SETTEMP = float(val)
    except:
        SETTEMP=30.0
    #return val
def updateerror(val):
    global ERROR
    val=val.replace(',','.')
    try:
        ERROR = float(val)
    except:
        ERROR = 0.5
        
def restart(event):
    global listoftemps, listoftimes,timeinit
    timeinit=time.time()
    listoftemps=[]
    listoftimes=[]
def start(event):
    restart(0)
    global startedflag
    startedflag=1
    control('start')
    return 0
def exit(event):
    global exitflag, startedflag
    if startedflag==1:
        control('start')
        startedflag=0
    exitflag=1
    return 0
def controlup(event):
    control('up')
def controlstart(event):
    control('start')
def controldown(event):
    control('down')
def controlstop(event):
    control('stop')

In [4]:
tempmax=55.5
tempmin=19.9
font = cv2.FONT_HERSHEY_SIMPLEX 
org = (50, 50) 
fontScale = 0.5
color = (255, 255, 255) 
thickness = 2
SETTEMP=36
drawing = False
point1 = (0,0)
point2 = (10,10)
exitflag=0
startedflag=0
ERROR=0.5

filename=easygui.enterbox(msg='Enter filename.', title=' ', default='tests')#, strip=True, image=None, root=None)
#filename='tests'
%matplotlib qt
matplotlib.use("Qt5Agg")
def mouse_drawing(event, x, y, flags, params):
    global point1, point2, drawing
    if event == cv2.EVENT_LBUTTONDOWN:
        if drawing is False:
            drawing = True
            point1 = (x, y)
        else:
            drawing = False
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing is True:
            point2 = (x, y)
cap = cv2.VideoCapture(0)
cv2.namedWindow("Frame")
cv2.setMouseCallback("Frame", mouse_drawing)
timeinit=time.time()
listoftemps=[]
listoftimes=[]
plt.ion()
vid_cod = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter("cam_video.mp4", vid_cod, 6.0, (1280, 960))
lasttime=time.time()

#BUTTONS
contrpauseax=plt.axes([0.1,0.9,0.1,0.04])
contrpauseaxb = Button(contrpauseax, 'PAUSE', color='violet', hovercolor='green')
contrpauseaxb.on_clicked(controlstop)
contrupax=plt.axes([0.3,0.9,0.1,0.04])
contrupaxb = Button(contrupax, 'UP', color='violet', hovercolor='green')
contrupaxb.on_clicked(controlup)
contrdownax=plt.axes([0.5,0.9,0.1,0.04])
contrdownaxb = Button(contrdownax, 'DOWN', color='violet', hovercolor='green')
contrdownaxb.on_clicked(controldown)
contrstopax=plt.axes([0.7,0.9,0.1,0.04])
contrstopaxb = Button(contrstopax, 'START/STOP', color='violet', hovercolor='green')
contrstopaxb.on_clicked(controlstart)
axcut = plt.axes([0.1, 0.08, 0.15, 0.04])
#maxsizeSlider = Slider(axcut, 'Settemp', 20, 60, SETTEMP)
#maxsizeSlider.on_changed(update)

textbox= TextBox(axcut, 'Settemp', initial='60')
#(axbox, 'Evaluate', initial=initial_text)
textbox.on_submit(update)


butax = plt.axes([0.1, 0.01, 0.20, 0.04])
bcut = Button(butax, 'RESTART', color='violet', hovercolor='green')
bcut.on_clicked(restart)
bstrtax = plt.axes([0.4, 0.01, 0.20, 0.04])
bstart = Button(bstrtax, 'START', color='violet', hovercolor='green')
bstart.on_clicked(start)
bexitax = plt.axes([0.7, 0.01, 0.20, 0.04])
bexit = Button(bexitax, 'EXIT', color='violet', hovercolor='green')
bexit.on_clicked(exit)
axerr = plt.axes([0.6, 0.08, 0.15, 0.04])
errbox=TextBox(axerr, 'Set error', initial='0.5')
errbox.on_submit(updateerror)
#errorSlider = Slider(axerr, 'Seterror', 0, 2, ERROR)
#errorSlider.on_changed(updateerror)

while (True):
    #plt.figure(2)
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray=frame.copy()
    gray=cv2.resize(gray, (1280, 960))
    #uppertemp=gray[25:95, 1119:1255]
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    if exitflag==1:
        break
    if point1 and point2:
        cv2.rectangle(gray, point1, point2, (255, 255, 255))
        avertemp=np.mean(gray[min(point1[1],point2[1]):max(point1[1],point2[1]),min(point1[0],point2[0]):max(point1[0],point2[0]),:])/255*(tempmax-tempmin)+tempmin
        gray=cv2.applyColorMap(gray, cv2.COLORMAP_JET)
        gray=cv2.rectangle(gray, (43,33), (216,60), (0,0,0),-1)
        gray = cv2.putText(gray, 'T '+str(avertemp)[:5]+' C' + "TSET= "+ str(SETTEMP)[:5], org, font,  fontScale, color, thickness, cv2.LINE_AA)
        if np.isnan(avertemp)==False:
            listoftemps.append(avertemp)
            listoftimes.append(time.time()-timeinit)
        axplot=plt.axes([0.2,0.2,0.6,0.6])
        axplot.plot(listoftimes,listoftemps)
        plt.draw()
        mypause(0.1)
        axplot.cla()
        plt.show()
        if startedflag==1:
            if avertemp<SETTEMP-ERROR:
                gray = cv2.putText(gray, 'HEAT', (50,100), font,  fontScale, (0,0,255), thickness, cv2.LINE_AA)
                if time.time()-lasttime>5:
                    control('up')
                    lasttime=time.time()
            if avertemp>SETTEMP+ERROR:
                gray = cv2.putText(gray, 'COOL', (50,100), font,  fontScale, (255,0,0), thickness, cv2.LINE_AA)
                if time.time()-lasttime>5:
                    control('down')
                    lasttime=time.time()
    output.write(gray)
    cv2.imshow("Frame", gray)
cap.release()
output.release()
#control('start')
cv2.destroyAllWindows()
plt.plot(listoftimes,listoftemps)
#plt.plot(listoftimes,listoftemps)
with open('savedcsvs/'+filename+'.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(listoftimes, listoftemps))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:99: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


FileNotFoundError: [Errno 2] No such file or directory: 'savedcsvs/drop2.csv'

In [ ]:
control('start')

In [5]:
with open('kargina/'+filename+'.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(zip(listoftimes, listoftemps))